# <font color='black'> Практическое занятие 2 </font>
## <font color='black'> Анализ панельных данных: часть 1. Модели с фиксированными эффектами (FE-models)</font>
Типичные панельные данные имеют как пространственное, так и временное измерение. К примеру, наблюдения для одной страны представлены за несколько лет. При такой структуре данных наивно было бы полагать, что условие о независимости наблюдений соблюдается. Наблюдения "внутри" одной пространственной единицы, разумеется, связаны. В этом случае мы не можем применять классическую линейную регрессию. В качестве альтернативы выступают широко применяемые модели с фиксированными эффектами. 
Кратко о данных, с которыми мы сегодня будем работать:
* country -  страна: массив содержит данные по 27 посткоммунистическим странам
* period - период: данные усреднены по пятилеткам начиная с 1989 г. 
* fh_polity - индекс демократии, полученный на основе показателей Freedom House, Polity. Непрерывная шкала от 0 до 10, где 10 означает максимальное значение уровня демократии. 
* state_capacity - индекс государственной состоятельности. Приведен к непрерывной шкале от 0 до 10, где 10 - наиболее высокое значение государственной состоятельности. 

Подгрузим необходимые библиотеки и откроем массив "lab2.dta". 

In [ ]:
import pandas as pd
import statsmodels.formula.api as statf 
import numpy as np
import numpy.linalg as la
import seaborn as sb 
!pip install linearmodels
from linearmodels import PanelOLS
from scipy import stats

In [ ]:
data = pd.read_stata("lab2.dta")
data.head(10)

В нашей первой модели мы учтем разные "стартовые" условия. Для этого добавим дамми-переменные на страну. Проинтерпретируйте оценки model1. Почему в данном случае мы получаем большой R-squared? 

In [ ]:
model1 = statf.ols(formula = 'fh_polity ~ state_capacity + C(country)', data = data).fit()
print(model1.summary())

Как Вы уже могли заметить выше, Python по умолчанию в качестве базовой категории (относительно которой происходит сравнение) выбирает первую пространственную единицу. При этом не всегда удобно интерпретировать относительно первой категории. Ниже рассмотрим, как можно изменить базовую категорию. Проинтерпретируйте оценки model1_2. 

In [ ]:
data['country1'] = pd.Categorical(data.country)
model1_2 = statf.ols(formula = 'fh_polity ~ state_capacity +C(country1, Treatment("Russia"))', data = data).fit()
print(model1_2.summary())

Сохраним предсказанные значения из модели. 

In [ ]:
data['ypredicted'] = model1.predict()
data.head(10)

На следующем шаге оценим модель с внутригрупповым преобразованием. Вспомните, в чем ее отличие от LSDV-модели (модели с дамми-переменными), почему часто используется именно такая спецификация с внутригрупповым преобразованием. Обратите внимание на то, что для того, чтобы в Python оценить FE-модели и RE-модели, нужно предварительно указать, какая переменная показывает пространственное измерение, а какая - временное. 
* Проинтерпретируйте оценки model2. 
* Проинтерпретируйте значения F-test for Poolability. 

In [ ]:
data1 = data.set_index(['country', 'period'])
model2 = PanelOLS.from_formula('fh_polity ~ state_capacity + EntityEffects', data = data1).fit()
print(model2)

Проверим, действительно ли FE-модель хорошо объясняет зависимую переменную. Оценим корреляцию зависимой переменной (наблюдаемого отклика) и предсказанного значения отклика по подгруппам. Если корреляция низкая (по модулю), то это говорит о том, что мы плохо объясняем моделью то, что происходит в данной группе (стране). Переоценим модель на подвыборке тех стран, для которых корреляцию по модулю больше 0.3 и посмотрим, сохранились ли наши результаты.  

In [ ]:
groups_data = pd.DataFrame(data.groupby('country')['ypredicted'].corr(data['fh_polity']))
def grouping(series): 
    if abs(series) < 0.3: return 0 
    elif abs(series) >= 0.3: return 1

groups_data['group'] = groups_data['ypredicted'].apply(grouping)
data_merged = pd.merge(data, groups_data, on = 'country')
data_merged.rename(columns = {"ypredicted_y":"corr"})

In [ ]:
high = data_merged[data_merged['group']==1]
low = data_merged[data_merged['group']==0]
model5 = statf.ols(formula = 'fh_polity ~ state_capacity + C(country)', data = high).fit()
print(model5.summary())